# Overview
The goal of this project is to predict the final score of an NBA playoff game based on the previous scoring activity of the teams who are playing on a given day. The code included is meant to be run the same day as the playoff game occuring.

## Description of Data and Rationale

### Current Season Regular Season Data
Once the post season begins, we will be able to use all of the current regular season games to see how to the individual teams in question scored throughout the season. 

### Current Regular Season Games Post Trade Deadline
Throughout a regular season trades can occur especially around the trade deadline. Post the trade deadline, the team is no longer able to be altered. As such, the games played after the trade deadline are more indicative of post season performance. 

### Current Season Games Against Each Other
Throughout the season, each team should play the other between 3-4 times. These games should be weighted higher inside of the predicition model as they are games played directly between the two teams in question. 

### Current Season Games Against Each Other Post Trade Deadline (if applicable)
Post the trade deadline, there is the potential that the teams that are in the post season have played against each other with their "finalized" roster. If these games exist, they should have be weighted higher than the current season games the two teams played against each other. 

### Post Season Games Against Each Other (if applicable)
As the post progresses, there will be more games played between the teams in question that we can use. These games should have the highest weight as they are the most recent data point(s) we have. 

## Additional Items to Consider Adding

### Injuries and Player Avaliability
Look into the games where key player(s) were unavaliable and compare them to the games when the key player (s) returned.

### Home/Away Performance
Teams tend to perform differently at home versus on the road.

### Pace of Play
Some teams play faster or slower in comparison to other teams. This will affect the overall score of the game.

In [7]:
#Install the nba_api to pull down nba stats
pip install nba_api

Note: you may need to restart the kernel to use updated packages.


In [8]:
#Importing Packages
import pandas as pd
import pprint
from nba_api.stats.endpoints import playercareerstats
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import boxscoretraditionalv2

In [9]:
#Hardcoding important season dates
season_start_date = '2023-10-24'
season_end_date = '2024-04-14'
trade_deadline_date = '2024-02-08'

In [10]:
#Pull the Game Information for Today's Games and returns the game id and home/away team ids
def get_todays_ids():
    games = scoreboard.ScoreBoard()
    todays_game_dict = games.get_dict()
    games_list = todays_game_dict['scoreboard']['games']
    team_id_list = []
    for game in games_list:
        team_id_dict = {
            'game_id' : game['gameId'],
            'home_team_name' : game['homeTeam']['teamName'],
            'home_team_abbreviation' : game['homeTeam']['teamTricode'],
            'home_team_id' : game['homeTeam']['teamId'],
            'away_team_name' : game['awayTeam']['teamName'],
            'away_team_abbreviation' : game['awayTeam']['teamTricode'],
            'away_team_id' : game['awayTeam']['teamId']
        }
        team_id_list.append(team_id_dict)
    return team_id_list

In [11]:
#Get today's game information
todays_games = get_todays_ids()

todays_games

[{'game_id': '0042300301',
  'home_team_name': 'Celtics',
  'home_team_abbreviation': 'BOS',
  'home_team_id': 1610612738,
  'away_team_name': 'Pacers',
  'away_team_abbreviation': 'IND',
  'away_team_id': 1610612754}]

# Current Season Regular Season Data

## Notes
- If a team particpated in the In-Season Tournament, those two teams will have played an 83 game season rather than an 82 game season. 

- From NBA.com: How does the In-Season Tournament affect the regular season? 
    - "Every team will still play an 82-game regular season. All In-Season Tournament games will count toward the         regular-season standings except the Championship, which will sit outside the regular season."
    
- 2023-12-09
    - Indiana Pacers vs LA Lakers

## Home Team

In [12]:
#Create a dictionary to store DataFrames for all regular season games for each home team listed
home_team_dfs = {}

In [13]:
#Loop through each game and create a DataFrame for each home team listed
for game in todays_games:
    home_team_name = game['home_team_name']
    home_team_id = game['home_team_id']
    
    #Pulling in the individual game information played by the home teams
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=home_team_id)
    home_team_games_df = gamefinder.get_data_frames()[0]
    
    #Filtering the DataFrame so we only see the Regular Season Games from this Season. 
    home_team_regular_season_games_df = home_team_games_df[
        (home_team_games_df['GAME_DATE'] >= season_start_date) & 
        (home_team_games_df['GAME_DATE'] <= season_end_date)
    ]
    
    #Reset the Index for the DataFrame so that they are 0 Indexed
    home_team_regular_season_games_df.reset_index(drop=True, inplace=True)
    
    #Store the DataFrame in the Dictionary
    home_team_dfs[home_team_name] = home_team_regular_season_games_df

In [14]:
#Grabs the home team DataFrame when the desired home team name is passed in

def get_home_team_dataframe(home_team_name):
    return home_team_dfs[home_team_name]

get_home_team_dataframe(todays_games[0]['home_team_name'])

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22023,1610612738,BOS,Boston Celtics,0022301186,2024-04-14,BOS vs. WAS,W,240,132,...,0.667,10,38,48,29,10,15,14,13,10.0
1,22023,1610612738,BOS,Boston Celtics,0022301173,2024-04-12,BOS vs. CHA,W,240,131,...,0.692,16,37,53,34,12,1,6,20,33.0
2,22023,1610612738,BOS,Boston Celtics,0022301167,2024-04-11,BOS vs. NYK,L,242,109,...,0.813,10,26,36,25,7,12,12,16,-9.0
3,22023,1610612738,BOS,Boston Celtics,0022301148,2024-04-09,BOS @ MIL,L,241,91,...,NaN,12,26,38,27,11,4,12,8,-13.0
4,22023,1610612738,BOS,Boston Celtics,0022301134,2024-04-07,BOS vs. POR,W,241,124,...,0.769,5,35,40,34,11,8,12,12,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,22023,1610612738,BOS,Boston Celtics,0022300136,2023-11-04,BOS @ BKN,W,240,124,...,0.852,10,40,50,22,4,6,11,17,10.0
78,22023,1610612738,BOS,Boston Celtics,0022300118,2023-11-01,BOS vs. IND,W,239,155,...,0.964,11,46,57,27,5,2,11,19,51.0
79,22023,1610612738,BOS,Boston Celtics,0022300103,2023-10-30,BOS @ WAS,W,243,126,...,0.714,15,36,51,31,11,6,17,21,19.0
80,22023,1610612738,BOS,Boston Celtics,0022300080,2023-10-27,BOS vs. MIA,W,239,119,...,0.684,16,39,55,20,7,6,15,19,8.0


## Away Team

In [15]:
#Create a dictionary to store DataFrames for all regular season games for each away team listed
away_team_dfs = {}

In [16]:
#Loop through each game and create a DataFrame for each home team listed
for game in todays_games:
    away_team_name = game['away_team_name']
    away_team_id = game['away_team_id']
    
    #Pulling in the individual game information played by the home teams
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=away_team_id)
    away_team_games_df = gamefinder.get_data_frames()[0]
    
    #Filtering the DataFrame so we only see the Regular Season Games from this Season. 
    away_team_regular_season_games_df = away_team_games_df[
        (away_team_games_df['GAME_DATE'] >= season_start_date) & 
        (away_team_games_df['GAME_DATE'] <= season_end_date)
    ]
    
    #Reset the Index for the DataFrame so that they are 0 Indexed
    away_team_regular_season_games_df.reset_index(drop=True, inplace=True)
    
    #Store the DataFrame in the Dictionary
    away_team_dfs[away_team_name] = away_team_regular_season_games_df

In [17]:
#Grabs the away team DataFrame when the desired away team name is passed in

def get_away_team_dataframe(away_team_name):
    return away_team_dfs[away_team_name]

get_away_team_dataframe(todays_games[0]['away_team_name'])

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22023,1610612754,IND,Indiana Pacers,0022301188,2024-04-14,IND vs. ATL,W,239,157,...,0.800,12,36,48,41,10,6,14,24,42.0
1,22023,1610612754,IND,Indiana Pacers,0022301174,2024-04-12,IND @ CLE,L,239,120,...,0.864,7,32,39,37,7,2,13,17,-9.0
2,22023,1610612754,IND,Indiana Pacers,0022301146,2024-04-09,IND @ TOR,W,240,140,...,0.696,15,29,44,33,9,8,13,23,17.0
3,22023,1610612754,IND,Indiana Pacers,0022301133,2024-04-07,IND vs. MIA,W,241,117,...,0.952,4,33,37,27,4,6,6,24,2.0
4,22023,1610612754,IND,Indiana Pacers,0022301116,2024-04-05,IND vs. OKC,W,241,126,...,0.905,8,31,39,30,8,5,13,23,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,22023,1610612754,IND,Indiana Pacers,0022300001,2023-11-03,IND vs. CLE,W,239,121,...,0.667,7,33,40,28,7,5,18,18,5.0
79,22023,1610612754,IND,Indiana Pacers,0022300118,2023-11-01,IND @ BOS,L,241,104,...,0.583,8,23,31,26,6,6,9,20,-51.0
80,22023,1610612754,IND,Indiana Pacers,0022300102,2023-10-30,IND vs. CHI,L,240,105,...,0.840,11,36,47,26,7,5,17,24,-7.0
81,22023,1610612754,IND,Indiana Pacers,0022300091,2023-10-28,IND @ CLE,W,240,125,...,0.824,11,37,48,36,3,4,7,20,12.0


# Current Regular Season Games Post Trade Deadline

## Home Team

In [18]:
#Create a dictionary to store DataFrames for all post trade deadline games for each home team listed
post_deadline_home_team_dfs = {}

In [19]:
#Loop through each game and create a DataFrame for each home team listed
for game in todays_games:
    post_deadline_home_team_name = game['home_team_name']
    post_deadline_home_team_id = game['home_team_id']
    
    #Pulling in the individual game information played by the home teams
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=post_deadline_home_team_id)
    post_deadline_home_team_games_df = gamefinder.get_data_frames()[0]
    
    #Filtering the DataFrame so we only see the Regular Season Games from this Season. 
    home_team_post_deadline_games_df = post_deadline_home_team_games_df[
        (post_deadline_home_team_games_df['GAME_DATE'] >= trade_deadline_date) & 
        (post_deadline_home_team_games_df['GAME_DATE'] <= season_end_date)
    ]
    
    #Reset the Index for the DataFrame so that they are 0 Indexed
    home_team_post_deadline_games_df.reset_index(drop=True, inplace=True)
    
    #Store the DataFrame in the Dictionary
    post_deadline_home_team_dfs[post_deadline_home_team_name] = home_team_post_deadline_games_df

In [20]:
#Grabs the post trade deadline home team DataFrame when the desired home team name is passed in

def get_post_deadline_home_team_dataframe(home_team_name):
    return post_deadline_home_team_dfs[home_team_name]

get_post_deadline_home_team_dataframe(todays_games[0]['home_team_name'])

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22023,1610612738,BOS,Boston Celtics,0022301186,2024-04-14,BOS vs. WAS,W,240,132,...,0.667,10,38,48,29,10,15,14,13,10.0
1,22023,1610612738,BOS,Boston Celtics,0022301173,2024-04-12,BOS vs. CHA,W,240,131,...,0.692,16,37,53,34,12,1,6,20,33.0
2,22023,1610612738,BOS,Boston Celtics,0022301167,2024-04-11,BOS vs. NYK,L,242,109,...,0.813,10,26,36,25,7,12,12,16,-9.0
3,22023,1610612738,BOS,Boston Celtics,0022301148,2024-04-09,BOS @ MIL,L,241,91,...,NaN,12,26,38,27,11,4,12,8,-13.0
4,22023,1610612738,BOS,Boston Celtics,0022301134,2024-04-07,BOS vs. POR,W,241,124,...,0.769,5,35,40,34,11,8,12,12,17.0
5,22023,1610612738,BOS,Boston Celtics,0022301118,2024-04-05,BOS vs. SAC,W,239,101,...,0.722,12,39,51,25,6,4,7,11,1.0
6,22023,1610612738,BOS,Boston Celtics,0022301105,2024-04-03,BOS vs. OKC,W,242,135,...,0.947,11,35,46,33,7,10,10,18,35.0
7,22023,1610612738,BOS,Boston Celtics,0022301087,2024-04-01,BOS @ CHA,W,239,118,...,0.813,11,40,51,28,3,3,10,15,14.0
8,22023,1610612738,BOS,Boston Celtics,0022301074,2024-03-30,BOS @ NOP,W,240,104,...,0.917,8,43,51,27,7,7,12,13,12.0
9,22023,1610612738,BOS,Boston Celtics,0022301060,2024-03-28,BOS @ ATL,L,267,122,...,0.857,9,34,43,28,7,6,12,15,-1.0


## Away Team

In [21]:
#Create a dictionary to store DataFrames for all post trade deadline games for each away team listed
post_deadline_away_team_dfs = {}

In [22]:
#Loop through each game and create a DataFrame for each away team listed
for game in todays_games:
    post_deadline_away_team_name = game['away_team_name']
    post_deadline_away_team_id = game['away_team_id']
    
    #Pulling in the individual game information played by the home teams
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=post_deadline_away_team_id)
    post_deadline_away_team_games_df = gamefinder.get_data_frames()[0]
    
    #Filtering the DataFrame so we only see the Regular Season Games from this Season. 
    away_team_post_deadline_games_df = post_deadline_away_team_games_df[
        (post_deadline_away_team_games_df['GAME_DATE'] >= trade_deadline_date) & 
        (post_deadline_away_team_games_df['GAME_DATE'] <= season_end_date)
    ]
    
    #Reset the Index for the DataFrame so that they are 0 Indexed
    away_team_post_deadline_games_df.reset_index(drop=True, inplace=True)
    
    #Store the DataFrame in the Dictionary
    post_deadline_away_team_dfs[post_deadline_away_team_name] = away_team_post_deadline_games_df

In [23]:
#Grabs the post trade deadline away team DataFrame when the desired away team name is passed in

def get_post_deadline_away_team_dataframe(away_team_name):
    return post_deadline_away_team_dfs[away_team_name]

get_post_deadline_away_team_dataframe(todays_games[0]['away_team_name'])

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22023,1610612754,IND,Indiana Pacers,0022301188,2024-04-14,IND vs. ATL,W,239,157,...,0.800,12,36,48,41,10,6,14,24,42.0
1,22023,1610612754,IND,Indiana Pacers,0022301174,2024-04-12,IND @ CLE,L,239,120,...,0.864,7,32,39,37,7,2,13,17,-9.0
2,22023,1610612754,IND,Indiana Pacers,0022301146,2024-04-09,IND @ TOR,W,240,140,...,0.696,15,29,44,33,9,8,13,23,17.0
3,22023,1610612754,IND,Indiana Pacers,0022301133,2024-04-07,IND vs. MIA,W,241,117,...,0.952,4,33,37,27,4,6,6,24,2.0
4,22023,1610612754,IND,Indiana Pacers,0022301116,2024-04-05,IND vs. OKC,W,241,126,...,0.905,8,31,39,30,8,5,13,23,14.0
5,22023,1610612754,IND,Indiana Pacers,0022301106,2024-04-03,IND @ BKN,L,240,111,...,0.818,19,26,45,27,8,9,9,21,-4.0
6,22023,1610612754,IND,Indiana Pacers,0022301089,2024-04-01,IND vs. BKN,W,240,133,...,0.824,14,33,47,35,6,9,9,21,22.0
7,22023,1610612754,IND,Indiana Pacers,0022301063,2024-03-29,IND vs. LAL,W,239,109,...,0.929,6,40,46,27,8,6,8,14,19.0
8,22023,1610612754,IND,Indiana Pacers,0022301054,2024-03-27,IND @ CHI,L,238,99,...,0.783,12,28,40,25,3,4,13,23,-26.0
9,22023,1610612754,IND,Indiana Pacers,0022301043,2024-03-25,IND @ LAC,W,240,133,...,0.727,7,27,34,28,7,3,8,21,17.0
